In [21]:
import platform

In [22]:
platform.python_version()

'3.11.3'

## 데이터 처리 모듈 설치 
pip install pydantic

# 1. 데이터클래스 

In [10]:
from dataclasses import dataclass


## 디스크립트 dataclass를 클래스에 데코레이터 처리

- __get__ 을 지정해서 타입을 지정해서 처리 

In [14]:
dir(dataclass)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [ ]:
@dataclass
class Person:
    name: str
    age: int
    profession: str

In [11]:
# 데이터 클래스의 인스턴스 생성
person1 = Person("Alice", 30, "Engineer")
person2 = Person("Bob", 35, "Manager")

In [12]:
# 속성에 접근
print(person1.name)  # Alice
print(person2.age)   # 35

Alice
35


In [13]:
# 데이터 클래스의 문자열 표현
print(person1)  
print(person2)  

Person(name='Alice', age=30, profession='Engineer')
Person(name='Bob', age=35, profession='Manager')


In [23]:
from datetime import datetime

# 2. pydantic 

- 파이썬 타입 어노테이션을 사용해서 데이터 유효성 검사와 설정 관리를 하는 라이브러리다. 

## 장점

- IDE plugin으로 제공되고 파이썬 타입 어노테이션과 유사하기 때문에 쉽게 사용할 수 있다.
- 비슷한 라이브러리 중에서 가장 빠르다.
- recursive model, typing 의 타입, validator 를 통해 복잡한 데이터 스키마를 정의하고, validation과 parsing을 할 수 있다.

In [2]:
from pydantic import BaseModel   


In [ ]:
## 데이터 클래스와 다르게 BaseModel을 상속해서 처리

- __fields__',
 '__fields_set__

In [8]:
dir(BaseModel)

['Config',
 '__abstractmethods__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose_class',
 '_enforce_dict_if_root',
 

In [3]:
class Model(BaseModel):     
    a: int     
    b: float     
    c: str 

In [5]:
m = Model(a=3.1415, b=' 2.72 ', c=123)

In [6]:
m

Model(a=3, b=2.72, c='123')

In [7]:
m.a

3

In [4]:
print(Model(a=3.1415, b=' 2.72 ', c=123).dict()) 

{'a': 3, 'b': 2.72, 'c': '123'}


In [17]:
class AdaptedModel(BaseModel):
    parent_attr: str

In [18]:
AdaptedModel.__fields__

{'parent_attr': ModelField(name='parent_attr', type=str, required=True)}

In [19]:
class TestClass(AdaptedModel):
    child_attr: str
        

In [20]:
TestClass.__fields__

{'parent_attr': ModelField(name='parent_attr', type=str, required=True),
 'child_attr': ModelField(name='child_attr', type=str, required=True)}

## ㅌ-1 자료형을 잘 못 처리하면 자동으로 수정해줌 

In [24]:
class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: datetime | None = None
    friends: list[int] = []


In [25]:
external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}

### 딕셔너리를 언팩처리해서 객체를 생성

In [26]:
user = User(**external_data)

In [27]:
user

User(id=123, signup_ts=datetime.datetime(2019, 6, 1, 12, 22), friends=[1, 2, 3], name='John Doe')

In [28]:
user.dict()

{'id': 123,
 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
 'friends': [1, 2, 3],
 'name': 'John Doe'}

In [29]:
print(user.id)

123


In [30]:
print(repr(user.signup_ts))

datetime.datetime(2019, 6, 1, 12, 22)


In [31]:
print(user.friends)

[1, 2, 3]


## ㅌ-2 예외처리 

In [32]:
from pydantic import ValidationError

try:
    User(signup_ts='broken', friends=[1, 2, 'not number'])
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "id"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "signup_ts"
    ],
    "msg": "invalid datetime format",
    "type": "value_error.datetime"
  },
  {
    "loc": [
      "friends",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]


## -3 데이터 클래스 사용하기 

In [35]:
from pydantic.dataclasses import dataclass


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    signup_ts: datetime = None


In [36]:
user = User(id='42', signup_ts='2032-06-21T12:00')
print(user)

User(id=42, name='John Doe', signup_ts=datetime.datetime(2032, 6, 21, 12, 0))
